<a href="https://colab.research.google.com/github/LeonardoCorreia08/TTs/blob/main/Atvd1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Célula Oculta
!pip install gTTS

In [ ]:
# @title Célula Oculta
!pip install pydub

In [ ]:
# @title Célula Oculta
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 42.4 MB/s eta 0:00:00


In [ ]:
# @title Célula Oculta
!pip install ffmpeg-python

In [ ]:
# @title Célula Oculta
!apt-get -qq install -y portaudio19-dev

Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120509 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package libportaudiocpp0:amd64.
Preparing to unpack .../libportaudiocpp0_19.6.0-1.1_amd64.deb ...
Unpacking libportaudiocpp0:amd64 (19.6.0-1.1) ...
Selecting previously unselected package portaudio19-dev:amd64.
Preparing to unpack .../portaudio19-dev_19.6.0-1.1_amd64.deb ...
Unpacking portaudio19-dev:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudiocpp0:amd64 (19.6.0-1.1) ...
Setting up portaudio19-dev:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...


In [ ]:
# @title Célula Oculta
!pip install pyaudio

  Using cached PyAudio-0.2.13.tar.gz (46 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.13-cp310-cp310-linux_x86_64.whl size=63870 sha256=02caf4ae0996d03f67defd2c983af238d33c1c83427f2c47d433f91ae797b12e
  Stored in directory: /root/.cache/pip/wheels/14/f1/c2/d102b4765a82c5a7bb273998dca7e4a53fc58e9a1a516fda81
Successfully built pyaudio


In [260]:
# @title Célula Oculta
!pip install nltk

In [235]:
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from pydub import AudioSegment
from google.colab import output
import os
import logging
from pydub import AudioSegment
from pydub.playback import play
import speech_recognition as sr
from base64 import b64decode
from googletrans import Translator
import nltk

# Baixe a lista de pontuações do nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Configuração do logger para evitar exibir o aviso RuntimeWarning
logging.getLogger("pydub.converter").setLevel(logging.ERROR)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [259]:
# @title Célula Oculta

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
# Função microfone
def record(sec=5):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'curs0.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio
  return f'/content/{file_name}'


# Função para adicionar a pontuação e sinais corretos ao texto traduzido
def add_punctuation(text):
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # Obter as tags de cada palavra para identificar a pontuação
    word_tags = nltk.pos_tag(words)

    # Dicionário para mapear a tag da pontuação para a pontuação correspondente
    punctuation_map = {
        "SYM": "",   # Símbolo
        "``": "\"",  # Aspas duplas esquerdas
        "''": "\"",  # Aspas duplas direitas
        "(": "(",    # Parênteses esquerdo
        ")": ")",    # Parênteses direito
        ",": ",",    # Vírgula
        ".": ".",    # Ponto final
        ":": ":",    # Dois pontos
        ";": ";",    # Ponto e vírgula
        "?": "?",    # Pergunta
        "!": "!"     # Exclamação
    }

    # Construir o texto com a pontuação correta
    punctuated_text = ""
    for word, tag in word_tags:
        if tag in punctuation_map:
            punctuated_text += punctuation_map[tag]
        punctuated_text += word + " "

    return punctuated_text

# Função para transcrever o áudio gravado em português e traduzir para inglês
def transcrever_e_traduzir(audio_file):
    recognizer = sr.Recognizer()
    translator = Translator()

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        texto_portugues = recognizer.recognize_google(audio_data, language="pt-BR")
        print("Texto em português reconhecido:", texto_portugues)

        # Traduzir o texto para inglês
        texto_ingles = translator.translate(texto_portugues, src="pt", dest="en")
        print("Texto traduzido para inglês:", texto_ingles.text)


    except sr.UnknownValueError:
        print("Não foi possível reconhecer o áudio. Por favor, tente novamente.")
        return None



In [248]:
# Variável texto inicialmente vazia
texto = ""

# Função para solicitar a opção ao usuário e retornar um número válido
def obter_opcao_valida():
    while True:
        print("Selecione uma opção:")
        print("0) Digitar uma frase")
        print("1) Frase 1: Olá, como vai? Como faço para saber se realizei a atividade certa?")
        print("2) Frase 2: Descubra como será o tempo e o clima em Recife.")
        print("3) Frase 3: Vamos nos divertir muito com essa biblioteca de áudio.")
        print("4) Frase 4: Python é uma linguagem de programação de alto nível.")
        print("5) Frase 5: A aprendizagem é um processo contínuo que ocorre durante toda a vida do indivíduo.")
        print("6) Estou lhe ouvindo, pode falar.")
        opcao = input("Opção: ")
        if opcao.isdigit() and 0 <= int(opcao) <= 6:
            return int(opcao)
        print("Número inválido. Por favor, digite um número de 0 a 6.")

# Obter opção válida
opcao = obter_opcao_valida()

# Defina a variável texto de acordo com a opção escolhida
if opcao == 0:
    texto = input("Digite a frase que deseja sintetizar em áudio: ")
elif opcao in range(1, 6):
    frases_portugues = [
        "Olá, como vai? Como faço para saber se realizei a atividade certa?",
        "Descubra como será o tempo e o clima em Recife.",
        "Vamos nos divertir muito com essa biblioteca de áudio.",
        "Python é uma linguagem de programação de alto nível.",
        "A aprendizagem é um processo contínuo que ocorre durante toda a vida do indivíduo."
    ]
    texto = frases_portugues[opcao - 1]
elif opcao == 6:
    print("Estou lhe ouvindo. Fale por alguns segundos...")
    # Grava o áudio do usuário por alguns segundos
    record_file = record()
    # Exibe o caminho do arquivo de áudio gravado
    print("Caminho do arquivo de áudio gravado:", audio_file_path)
    # Se ocorreu algum erro na gravação do áudio, encerra o programa
    if not record_file:
        exit()

    # Exibe o áudio gravado
    display(Audio(record_file, autoplay=False))


Selecione uma opção:
0) Digitar uma frase
1) Frase 1: Olá, como vai? Como faço para saber se realizei a atividade certa?
2) Frase 2: Descubra como será o tempo e o clima em Recife.
3) Frase 3: Vamos nos divertir muito com essa biblioteca de áudio.
4) Frase 4: Python é uma linguagem de programação de alto nível.
5) Frase 5: A aprendizagem é um processo contínuo que ocorre durante toda a vida do indivíduo.
6) Estou lhe ouvindo, pode falar.
Opção: 5


In [249]:
# Salve o áudio gerado como curso.mp3
tts = gTTS(text=texto, lang='pt')
tts.save("curs0.mp3")

In [250]:
# Utilize a função Audio para tocar o arquivo mp3 salvo
Audio("curs0.mp3", autoplay=True)

In [251]:
# Converte o arquivo gravado (MP3) para WAV
converted_wav_file = "curs0.wav"
convert_mp3_to_wav(record_file, converted_wav_file)

In [252]:
    # Define a variável texto como a transcrição do áudio gravado
    recognizer = sr.Recognizer()
    with sr.AudioFile(converted_wav_file) as source:
        audio_data = recognizer.record(source)

    try:
        texto = recognizer.recognize_google(audio_data, language="pt-BR")
        print("Texto reconhecido:", texto)
    except sr.UnknownValueError:
        print("Não foi possível reconhecer o áudio. Por favor, tente novamente.")
        exit()

Texto reconhecido: aprendizagem é um processo contínuo que ocorre durante toda a vida do indivíduo


In [255]:
# Chamar a função para transcrever e traduzir o áudio
transcrever_e_traduzir(converted_wav_file)

Texto em português reconhecido: aprendizagem é um processo contínuo que ocorre durante toda a vida do indivíduo
Texto traduzido para inglês: learning is a continuous process that occurs throughout the individual's life
